In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.DataFrame(pd.read_csv('../data/model_data2.csv'))
df.median_year_built = df.median_year_built.str.replace('-','')
df.median_year_built = df.median_year_built.astype('int')
df.head()

,Unnamed: 0,county_code,year,hpi_x,one_year,two_years,three_years,four_years,five_years,EMP,...,ten19_units,tenure,three4_units,three_rooms,twenty49_units,two_rooms,two_units,under_18,units_total,vacant
0,0,10001,2010,162.912258,-0.066988,-0.093230,-0.094294,-0.079433,-0.055848,46405,...,2690,57396,1144,12635,940,3992,752,39683,63563,6167
1,1,10001,2011,151.999032,-0.028126,-0.029266,-0.013338,0.011940,0.075964,48100,...,2777,57629,1162,12421,985,4045,953,40050,64616,6987
2,2,10001,2012,147.723871,-0.001173,0.015216,0.041226,0.107103,0.113881,47883,...,2997,57816,1202,12507,1102,4241,825,40375,65394,7578
3,3,10001,2013,147.550645,0.016408,0.042448,0.108403,0.115188,0.156970,48071,...,2836,58524,1323,12781,1104,4009,836,40384,65924,7400
4,4,10001,2014,149.971613,0.025620,0.090510,0.097186,0.138293,NaN,49311,...,3054,59142,1289,13164,1144,3627,773,40488,66652,7510


In [3]:
bins5 = [-1,0.137-2*0.155, 0.137-0.155, 0.137, 0.137+0.155,np.inf]
bins4 = [-1, 0.108-2*0.131, 0.108-0.131, 0.108,0.108+0.131, 5 ]
bins3 = [-1, 0.077-2*0.109,0.077-0.109, 0.077,0.077+0.109, 5 ]
bins2 = [-1, 0.048-2*0.086,0.048-0.086, 0.048,0.048+0.086, 5 ]
bins1 = [-1,0.018-2*0.059, 0.018-0.059, 0.018, 0.018+0.059, 5 ]
labels = [-2,-1, 0, 1, 2]


df['five_years_bin']=pd.cut(df['five_years'], bins=bins5, labels=labels)
df['four_years_bin']=pd.cut(df['four_years'], bins=bins4, labels=labels)
df['three_years_bin']=pd.cut(df['three_years'], bins=bins3, labels=labels)
df['two_years_bin']=pd.cut(df['two_years'], bins=bins2, labels=labels)
df['one_year_bin']=pd.cut(df['one_year'], bins=bins1, labels=labels)

df.five_years_bin.count()

10282

In [4]:
df = df[[
    'previous_one', 
    'previous_two',
    'previous_three',
    'previous_four',
    'previous_five',
    'pop_chg_yoy',
    'EMP',
    'ESTAB',
    'PAYANN',
    'one_year_emp',
    'two_years_emp',
    'three_years_emp',
    'four_years_emp',
    'five_years_emp',
    'one_year_estab',
    'two_years_estab',
    'three_years_estab',
    'four_years_estab',
    'five_years_estab',
    'one_year_pay',
    'two_years_pay',
    'three_years_pay',
    'four_years_pay',
    'five_years_pay',
    'single_units',
    'total_units',
    'one_year_pop',
    'two_years_pop',
    'three_years_pop',
    'four_years_pop', 
    'five_years_pop',
    'hh_income', 
    'one_year_inc',
    'two_years_inc',
    'three_years_inc',
    'four_years_inc',
    'five_years_inc',
    'aggregate_rent',
    'aggregate_rooms',
    'bedrooms_total',
    'born_population',
    'family_household',
    'fiftyplus_units',
    'five9_units',
    'fiveplus_rooms',
    'foreign_born',
    'four_rooms',
    'geographic_mobility',
    'household_type',
    'housing_units',
    'married_household', 
    'median_age', 
    'median_rent',
    'median_rooms',
    'median_year_built',
    'native_born',
    'naturalized',
    'nonfamily_household',
    'occupancy_status_total',
    'occupied', 
    'one_room',
    'owner_tenure',
    'population',
    'poverty_count',
    'renter_tenure',
    'single_attached',
    'single_detached', 
    'ten19_units',
    'tenure',
    'three4_units',
    'three_rooms',
    'twenty49_units',
    'two_rooms',
    'two_units',
    'under_18',
    'units_total',
    'vacant',
    'five_years_bin',
    'four_years_bin',
    'three_years_bin',
    'two_years_bin',
    'one_year_bin',
#     'five_years',
#     'four_years',
#     'three_years',
#     'two_years',
#     'one_year'
          ]]

In [5]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

In [6]:
X = df.drop(['five_years_bin','four_years_bin','three_years_bin','two_years_bin','one_year_bin'], axis=1)
ys = ['five_years_bin','four_years_bin','three_years_bin','two_years_bin','one_year_bin']
print(X.shape)

(5052, 77)


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

y = df[[ys[4]]]
X_scaler = StandardScaler().fit(X)
X_scaled = X_scaler.transform(X)
y_scaler = StandardScaler().fit(y)
y_scaled = y_scaler.transform(y)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, random_state=73)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# # Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


C:\Users\nilay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\nilay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=77, activation='linear', input_dim=77))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=5, activation ='tanh'))
model.add(Dense(units=5, activation='softmax'))

In [30]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(
    X_train,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=20
)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [31]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.10438665956471142, Accuracy: 0.6104513064368982


In [ ]:
model.predict_proba(X_test)

In [32]:
model.save('classification_model1.h5')

In [ ]:
def load_model():
    global model1
    model1 = keras.models.load_model("classification_model.h5")

In [ ]:
import keras
load_model()

In [ ]:
a = np.array([X.iloc[0].values])

In [ ]:
a

In [ ]:
model1.predict_proba(a)

In [ ]:
X_train